![UNIR](https://www.unir.net/wp-content/uploads/2014/10/logo.png)
### Máster en Inteligencia Artificial. 
**Asignatura:** _Aprendizaje Automático._

**Equipo 14:** Sergio Merino,  Laia García, Luisa Sánchez y Miguel Á. de Frutos

**Fecha:** _17 Enero 2020_

---

# ACTIVIDAD 2: Árboles y Random-Forest para regresión.

**Objetivo:** Aplicar al conjunto de datos proporcionado los métodos o algoritmos Árboles y Random Forest para predecir (regresión) el número de ventas de bicicletas por hora..
___

## 1. Análisis descriptivo de los datos.

In [ ]:
import pandas as pd
bicis = pd.read_csv("./datos/hour.csv") # Importamos el data set
bicis.head()   

In [49]:
bicis.shape

(17379, 17)

Analizo la información disponible:

In [50]:
bicis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
instant       17379 non-null int64
dteday        17379 non-null object
season        17379 non-null int64
yr            17379 non-null int64
mnth          17379 non-null int64
hr            17379 non-null int64
holiday       17379 non-null int64
weekday       17379 non-null int64
workingday    17379 non-null int64
weathersit    17379 non-null int64
temp          17379 non-null float64
atemp         17379 non-null float64
hum           17379 non-null float64
windspeed     17379 non-null float64
casual        17379 non-null int64
registered    17379 non-null int64
cnt           17379 non-null int64
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


## 2. Tratamiento de datos: Missing y Redundancia.

- ***Missing:*** Analizamos si faltan datos o hay navores no numéricos.

In [51]:
print(bicis.isnull().any())

instant       False
dteday        False
season        False
yr            False
mnth          False
hr            False
holiday       False
weekday       False
workingday    False
weathersit    False
temp          False
atemp         False
hum           False
windspeed     False
casual        False
registered    False
cnt           False
dtype: bool


- **Redundancias:** Elimino columnas no útiles o problemáticas:

In [52]:
bicis= bicis.drop(['casual' , 'registered','atemp','instant','dteday', 'season', 'holiday'], axis=1)
bicis.head()  

,season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt
0,1,1,0,0,6,0,1,0.24,0.81,0.0,16
1,1,1,1,0,6,0,1,0.22,0.80,0.0,40
2,1,1,2,0,6,0,1,0.22,0.80,0.0,32
3,1,1,3,0,6,0,1,0.24,0.75,0.0,13
4,1,1,4,0,6,0,1,0.24,0.75,0.0,1


## 3. Tratamiento de variables categóricas.

## 4. Determinar el conjunto de modelización y el de validación.

## 5. Identificar qué métricas de evaluación se utilizarán.

## 6. Implementación de ambos algoritmos.

## 7. Comparar los resultados obtenidos.

## 8. Otras consideraciones.

## 9. Repetir el ejecicio emplando un algoritmo de clasificación